In [17]:
from llm.context.llm_context import LLMContext
from llm.strategy.big_llama_model import BigLlamaModel
from llm.strategy.big_llama3_1_model import BigLlama3_1Model
from llm.strategy.small_llama_model import SmallLlamaModel
from llm.strategy.big_mistral_model import BigMistralModel
from llm.strategy.haiku3_model import Haiku3Model
from llm.strategy.sonet3_model import Sonet3Model
from llm.strategy.opus3_model import OpusModel
from llm.strategy.sonet3_5_model import Sonet3_5Model

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from tqdm import tqdm
import os
import numpy as np 
import pandas as pd
import re
import warnings
import spacy

In [2]:
PATH = './data/processed/txt'

In [3]:
def get_text_and_masked_carmen(name):
    filename_result = f'./data/processed/masked/{name}'

    with open(filename_result, 'r') as archivo:
        text_masked = archivo.read()

    csv_data = f'./data/processed/ann/{name.split(".")[0]}.csv'
    result_dict = {}
    df = pd.read_csv((csv_data), header=None)
    for index, row in df.iterrows():
        result_dict[row[3]] = row[0]
    return [text_masked, result_dict]

# METRICS

In [4]:
def compute_classification_agreement(dict1, dict2):
    # Look for key disagreement
    keys_not_present_in_dict2 = list(set(dict1.keys()) - set(dict2.keys()))
    
    # look for misclassifications
    disagreement_in_values = {}
    for key in dict1.keys() & dict2.keys():
        if dict1[key] != dict2[key]:
            disagreement_in_values[key] = (dict1[key], dict2[key])

    return disagreement_in_values, keys_not_present_in_dict2

# Loop

In [5]:
def store_metrics(real, predicted,fails, miss, metrics_data):
    metrics_data["total"] = len(real.keys())
    metrics_data["correct"] = len(real.keys()) - len(fails.keys())
    metrics_data["precision"] = 100 - (len(fails.keys()) * 100) / len(real.keys())
    metrics_data["real"] = real
    metrics_data["predicted"] = predicted
    metrics_data["fails"] = fails
    metrics_data["miss"] = miss
    return metrics_data

In [6]:
def save_metrics(name_model, list_data):
    list_data = pd.DataFrame(list_data)
    list_data.to_csv(f'data/metrics/second_step/{name_model}.csv')

In [7]:
import json
def anonimized(llm=None, name_model="", data=None):
    counter = 0
    context = LLMContext(llm)
    list_data  = []
    for filename in sorted(os.listdir(PATH)):
        print(filename)
        metrics_data = {}
        metrics_data["filename"] = filename
        try:
            [text_masked, array_ground_truth] = get_text_and_masked_carmen(filename)
            data["user"] = text_masked
            text_generated = context.generate_response(data)
            array_of_dicts = json.loads(text_generated)
            merged_dict = {}
            for d in array_of_dicts:
                merged_dict.update(d)
            diferencias, unicas_dict1 = compute_classification_agreement(array_ground_truth, merged_dict)
            metrics_data = store_metrics(array_ground_truth, merged_dict, diferencias, unicas_dict1, metrics_data)
            list_data.append(metrics_data)
            counter += 1
            if counter >= 100:
                break
        except Exception as e:
            print("Error")
            print(e)
            continue
    save_metrics(name_model, list_data)

In [8]:
data = {}
data["system"] = """
You are a clasification tool. 
You have the ability to identify quasi-identifying attributes which are in [**  **] simbols.
Then you will return an array with the anonymized attributes in the order they appear in the text.

Las posibles etiquetas son:
- 'FAMILIARES_SUJETO_ASISTENCIA'
- 'FECHAS'
- 'NUMERO_TELEFONO'
- 'OTROS_SUJETO_ASISTENCIA'
- 'INSTITUCION'
- 'NUMERO_IDENTIF'
- 'EDAD_SUJETO_ASISTENCIA'
- 'HOSPITAL'
- 'PAIS'
- 'TERRITORIO'
- 'NOMBRE_PERSONAL_SANITARIO'
- 'PROFESION'
- 'ID_SUJETO_ASISTENCIA'
- 'CENTRO_SALUD'
- 'SEXO_SUJETO_ASISTENCIA'
- 'CALLE'
- 'ID_CONTACTO_ASISTENCIAL'
- 'URL_WEB'


Examples:
Paciente de [**41 años**], [**albañil**], con dolor lumbar crónico y problemas de movilidad. = [{"41 años": "EDAD_SUJETO_ASISTENCIA"}, {"albañil": "PROFESION"}]

Do not comment anything else. Just return the array.

"""

# Function to get labels

In [9]:
# import pandas as pd
# import os
# PATH = './data/processed/ann'
# array_options = []
# for filename in sorted(os.listdir(PATH)):
#     filename_result = f'./data/processed/ann/{filename}'
#     data = pd.read_csv(filename_result)
#     array_options.append(data.iloc[:, 0])

# combined_options = [item for sublist in array_options for item in sublist]
# unique_options_ordered = list(dict.fromkeys(combined_options))
# unique_options_ordered

In [10]:
anonimized(llm=BigLlamaModel(), name_model="big_llama3", data=data)

CARMEN-I_CC_1.txt
CARMEN-I_CC_2.txt
CARMEN-I_CC_3.txt
CARMEN-I_CC_4.txt
CARMEN-I_CC_5.txt
CARMEN-I_IA_ANTECEDENTES_1.txt
CARMEN-I_IA_ANTECEDENTES_10.txt
CARMEN-I_IA_ANTECEDENTES_100.txt
CARMEN-I_IA_ANTECEDENTES_101.txt
CARMEN-I_IA_ANTECEDENTES_102.txt
CARMEN-I_IA_ANTECEDENTES_103.txt
CARMEN-I_IA_ANTECEDENTES_104.txt
CARMEN-I_IA_ANTECEDENTES_105.txt
CARMEN-I_IA_ANTECEDENTES_106.txt
CARMEN-I_IA_ANTECEDENTES_107.txt
CARMEN-I_IA_ANTECEDENTES_108.txt
CARMEN-I_IA_ANTECEDENTES_109.txt
CARMEN-I_IA_ANTECEDENTES_11.txt
CARMEN-I_IA_ANTECEDENTES_110.txt
CARMEN-I_IA_ANTECEDENTES_111.txt
CARMEN-I_IA_ANTECEDENTES_112.txt
CARMEN-I_IA_ANTECEDENTES_113.txt
CARMEN-I_IA_ANTECEDENTES_114.txt
CARMEN-I_IA_ANTECEDENTES_115.txt
CARMEN-I_IA_ANTECEDENTES_116.txt
CARMEN-I_IA_ANTECEDENTES_117.txt
CARMEN-I_IA_ANTECEDENTES_118.txt
CARMEN-I_IA_ANTECEDENTES_119.txt
CARMEN-I_IA_ANTECEDENTES_12.txt
CARMEN-I_IA_ANTECEDENTES_120.txt
CARMEN-I_IA_ANTECEDENTES_121.txt
CARMEN-I_IA_ANTECEDENTES_122.txt
CARMEN-I_IA_ANTECEDENTES

In [11]:
anonimized(llm=BigLlama3_1Model(), name_model="big_llama3_1", data=data)

CARMEN-I_CC_1.txt
CARMEN-I_CC_2.txt
CARMEN-I_CC_3.txt
CARMEN-I_CC_4.txt
CARMEN-I_CC_5.txt
CARMEN-I_IA_ANTECEDENTES_1.txt
CARMEN-I_IA_ANTECEDENTES_10.txt
CARMEN-I_IA_ANTECEDENTES_100.txt
CARMEN-I_IA_ANTECEDENTES_101.txt
CARMEN-I_IA_ANTECEDENTES_102.txt
CARMEN-I_IA_ANTECEDENTES_103.txt
CARMEN-I_IA_ANTECEDENTES_104.txt
CARMEN-I_IA_ANTECEDENTES_105.txt
CARMEN-I_IA_ANTECEDENTES_106.txt
CARMEN-I_IA_ANTECEDENTES_107.txt
CARMEN-I_IA_ANTECEDENTES_108.txt
CARMEN-I_IA_ANTECEDENTES_109.txt
CARMEN-I_IA_ANTECEDENTES_11.txt
CARMEN-I_IA_ANTECEDENTES_110.txt
CARMEN-I_IA_ANTECEDENTES_111.txt
CARMEN-I_IA_ANTECEDENTES_112.txt
CARMEN-I_IA_ANTECEDENTES_113.txt
CARMEN-I_IA_ANTECEDENTES_114.txt
CARMEN-I_IA_ANTECEDENTES_115.txt
CARMEN-I_IA_ANTECEDENTES_116.txt
CARMEN-I_IA_ANTECEDENTES_117.txt
CARMEN-I_IA_ANTECEDENTES_118.txt
CARMEN-I_IA_ANTECEDENTES_119.txt
CARMEN-I_IA_ANTECEDENTES_12.txt
CARMEN-I_IA_ANTECEDENTES_120.txt
CARMEN-I_IA_ANTECEDENTES_121.txt
CARMEN-I_IA_ANTECEDENTES_122.txt
CARMEN-I_IA_ANTECEDENTES

In [12]:
anonimized(llm=SmallLlamaModel(), name_model="small_llam", data=data)

CARMEN-I_CC_1.txt
CARMEN-I_CC_2.txt
Error
Expecting property name enclosed in double quotes: line 1 column 4 (char 3)
CARMEN-I_CC_3.txt
CARMEN-I_CC_4.txt
CARMEN-I_CC_5.txt
CARMEN-I_IA_ANTECEDENTES_1.txt
Error
Expecting property name enclosed in double quotes: line 1 column 4 (char 3)
CARMEN-I_IA_ANTECEDENTES_10.txt
Error
Expecting ',' delimiter: line 1 column 260 (char 259)
CARMEN-I_IA_ANTECEDENTES_100.txt
Error
Expecting property name enclosed in double quotes: line 1 column 4 (char 3)
CARMEN-I_IA_ANTECEDENTES_101.txt
Error
Expecting property name enclosed in double quotes: line 1 column 4 (char 3)
CARMEN-I_IA_ANTECEDENTES_102.txt
Error
Expecting property name enclosed in double quotes: line 1 column 4 (char 3)
CARMEN-I_IA_ANTECEDENTES_103.txt
CARMEN-I_IA_ANTECEDENTES_104.txt
CARMEN-I_IA_ANTECEDENTES_105.txt
Error
Expecting property name enclosed in double quotes: line 1 column 4 (char 3)
CARMEN-I_IA_ANTECEDENTES_106.txt
CARMEN-I_IA_ANTECEDENTES_107.txt
Error
Expecting ':' delimiter: 

In [13]:
anonimized(llm=BigMistralModel(), name_model="big_mistral", data=data)

CARMEN-I_CC_1.txt
CARMEN-I_CC_2.txt
Error
Unterminated string starting at: line 3 column 2 (char 25)
CARMEN-I_CC_3.txt
Error
Unterminated string starting at: line 3 column 2 (char 24)
CARMEN-I_CC_4.txt
CARMEN-I_CC_5.txt
CARMEN-I_IA_ANTECEDENTES_1.txt
CARMEN-I_IA_ANTECEDENTES_10.txt
CARMEN-I_IA_ANTECEDENTES_100.txt
Error
Unterminated string starting at: line 8 column 2 (char 171)
CARMEN-I_IA_ANTECEDENTES_101.txt
Error
Invalid \escape: line 8 column 53 (char 254)
CARMEN-I_IA_ANTECEDENTES_102.txt
CARMEN-I_IA_ANTECEDENTES_103.txt
Error
Unterminated string starting at: line 6 column 2 (char 120)
CARMEN-I_IA_ANTECEDENTES_104.txt
CARMEN-I_IA_ANTECEDENTES_105.txt
Error
Unterminated string starting at: line 5 column 2 (char 119)
CARMEN-I_IA_ANTECEDENTES_106.txt
CARMEN-I_IA_ANTECEDENTES_107.txt
CARMEN-I_IA_ANTECEDENTES_108.txt
CARMEN-I_IA_ANTECEDENTES_109.txt
Error
Unterminated string starting at: line 5 column 2 (char 103)
CARMEN-I_IA_ANTECEDENTES_11.txt
Error
Unterminated string starting at: l

In [14]:
anonimized(llm=Haiku3Model(), name_model="haiku_3", data=data)

CARMEN-I_CC_1.txt
CARMEN-I_CC_2.txt
CARMEN-I_CC_3.txt
CARMEN-I_CC_4.txt
CARMEN-I_CC_5.txt
CARMEN-I_IA_ANTECEDENTES_1.txt
CARMEN-I_IA_ANTECEDENTES_10.txt
CARMEN-I_IA_ANTECEDENTES_100.txt
CARMEN-I_IA_ANTECEDENTES_101.txt
CARMEN-I_IA_ANTECEDENTES_102.txt
CARMEN-I_IA_ANTECEDENTES_103.txt
CARMEN-I_IA_ANTECEDENTES_104.txt
CARMEN-I_IA_ANTECEDENTES_105.txt
CARMEN-I_IA_ANTECEDENTES_106.txt
CARMEN-I_IA_ANTECEDENTES_107.txt
CARMEN-I_IA_ANTECEDENTES_108.txt
CARMEN-I_IA_ANTECEDENTES_109.txt
CARMEN-I_IA_ANTECEDENTES_11.txt
CARMEN-I_IA_ANTECEDENTES_110.txt
CARMEN-I_IA_ANTECEDENTES_111.txt
CARMEN-I_IA_ANTECEDENTES_112.txt
CARMEN-I_IA_ANTECEDENTES_113.txt
CARMEN-I_IA_ANTECEDENTES_114.txt
CARMEN-I_IA_ANTECEDENTES_115.txt
CARMEN-I_IA_ANTECEDENTES_116.txt
CARMEN-I_IA_ANTECEDENTES_117.txt
CARMEN-I_IA_ANTECEDENTES_118.txt
CARMEN-I_IA_ANTECEDENTES_119.txt
CARMEN-I_IA_ANTECEDENTES_12.txt
CARMEN-I_IA_ANTECEDENTES_120.txt
CARMEN-I_IA_ANTECEDENTES_121.txt
CARMEN-I_IA_ANTECEDENTES_122.txt
CARMEN-I_IA_ANTECEDENTES

In [15]:
anonimized(llm=Sonet3Model(), name_model="sonet3", data=data)

CARMEN-I_CC_1.txt
CARMEN-I_CC_2.txt
CARMEN-I_CC_3.txt
CARMEN-I_CC_4.txt
CARMEN-I_CC_5.txt
CARMEN-I_IA_ANTECEDENTES_1.txt
CARMEN-I_IA_ANTECEDENTES_10.txt
CARMEN-I_IA_ANTECEDENTES_100.txt
CARMEN-I_IA_ANTECEDENTES_101.txt
CARMEN-I_IA_ANTECEDENTES_102.txt
CARMEN-I_IA_ANTECEDENTES_103.txt
CARMEN-I_IA_ANTECEDENTES_104.txt
CARMEN-I_IA_ANTECEDENTES_105.txt
CARMEN-I_IA_ANTECEDENTES_106.txt
CARMEN-I_IA_ANTECEDENTES_107.txt
CARMEN-I_IA_ANTECEDENTES_108.txt
CARMEN-I_IA_ANTECEDENTES_109.txt
CARMEN-I_IA_ANTECEDENTES_11.txt
CARMEN-I_IA_ANTECEDENTES_110.txt
CARMEN-I_IA_ANTECEDENTES_111.txt
CARMEN-I_IA_ANTECEDENTES_112.txt
CARMEN-I_IA_ANTECEDENTES_113.txt
CARMEN-I_IA_ANTECEDENTES_114.txt
CARMEN-I_IA_ANTECEDENTES_115.txt
CARMEN-I_IA_ANTECEDENTES_116.txt
CARMEN-I_IA_ANTECEDENTES_117.txt
CARMEN-I_IA_ANTECEDENTES_118.txt
CARMEN-I_IA_ANTECEDENTES_119.txt
CARMEN-I_IA_ANTECEDENTES_12.txt
CARMEN-I_IA_ANTECEDENTES_120.txt
CARMEN-I_IA_ANTECEDENTES_121.txt
CARMEN-I_IA_ANTECEDENTES_122.txt
CARMEN-I_IA_ANTECEDENTES

In [18]:
anonimized(llm=OpusModel(), name_model="opus3", data=data)

CARMEN-I_CC_1.txt
CARMEN-I_CC_2.txt
CARMEN-I_CC_3.txt
CARMEN-I_CC_4.txt
CARMEN-I_CC_5.txt
CARMEN-I_IA_ANTECEDENTES_1.txt
CARMEN-I_IA_ANTECEDENTES_10.txt
CARMEN-I_IA_ANTECEDENTES_100.txt
CARMEN-I_IA_ANTECEDENTES_101.txt
CARMEN-I_IA_ANTECEDENTES_102.txt
CARMEN-I_IA_ANTECEDENTES_103.txt
CARMEN-I_IA_ANTECEDENTES_104.txt
CARMEN-I_IA_ANTECEDENTES_105.txt
CARMEN-I_IA_ANTECEDENTES_106.txt
CARMEN-I_IA_ANTECEDENTES_107.txt
CARMEN-I_IA_ANTECEDENTES_108.txt
CARMEN-I_IA_ANTECEDENTES_109.txt
CARMEN-I_IA_ANTECEDENTES_11.txt
CARMEN-I_IA_ANTECEDENTES_110.txt
CARMEN-I_IA_ANTECEDENTES_111.txt
CARMEN-I_IA_ANTECEDENTES_112.txt
CARMEN-I_IA_ANTECEDENTES_113.txt
CARMEN-I_IA_ANTECEDENTES_114.txt
CARMEN-I_IA_ANTECEDENTES_115.txt
CARMEN-I_IA_ANTECEDENTES_116.txt
CARMEN-I_IA_ANTECEDENTES_117.txt
CARMEN-I_IA_ANTECEDENTES_118.txt
CARMEN-I_IA_ANTECEDENTES_119.txt
CARMEN-I_IA_ANTECEDENTES_12.txt
CARMEN-I_IA_ANTECEDENTES_120.txt
CARMEN-I_IA_ANTECEDENTES_121.txt
CARMEN-I_IA_ANTECEDENTES_122.txt
CARMEN-I_IA_ANTECEDENTES

In [19]:
anonimized(llm=Sonet3_5Model(), name_model="sonet3_5", data=data)

CARMEN-I_CC_1.txt
CARMEN-I_CC_2.txt
CARMEN-I_CC_3.txt
CARMEN-I_CC_4.txt
CARMEN-I_CC_5.txt
CARMEN-I_IA_ANTECEDENTES_1.txt
CARMEN-I_IA_ANTECEDENTES_10.txt
CARMEN-I_IA_ANTECEDENTES_100.txt
CARMEN-I_IA_ANTECEDENTES_101.txt
CARMEN-I_IA_ANTECEDENTES_102.txt
CARMEN-I_IA_ANTECEDENTES_103.txt
CARMEN-I_IA_ANTECEDENTES_104.txt
CARMEN-I_IA_ANTECEDENTES_105.txt
CARMEN-I_IA_ANTECEDENTES_106.txt
CARMEN-I_IA_ANTECEDENTES_107.txt
CARMEN-I_IA_ANTECEDENTES_108.txt
CARMEN-I_IA_ANTECEDENTES_109.txt
CARMEN-I_IA_ANTECEDENTES_11.txt
CARMEN-I_IA_ANTECEDENTES_110.txt
CARMEN-I_IA_ANTECEDENTES_111.txt
CARMEN-I_IA_ANTECEDENTES_112.txt
CARMEN-I_IA_ANTECEDENTES_113.txt
CARMEN-I_IA_ANTECEDENTES_114.txt
CARMEN-I_IA_ANTECEDENTES_115.txt
CARMEN-I_IA_ANTECEDENTES_116.txt
CARMEN-I_IA_ANTECEDENTES_117.txt
CARMEN-I_IA_ANTECEDENTES_118.txt
CARMEN-I_IA_ANTECEDENTES_119.txt
CARMEN-I_IA_ANTECEDENTES_12.txt
CARMEN-I_IA_ANTECEDENTES_120.txt
CARMEN-I_IA_ANTECEDENTES_121.txt
CARMEN-I_IA_ANTECEDENTES_122.txt
CARMEN-I_IA_ANTECEDENTES

# Precision

In [23]:
OpusModel_data = pd.read_csv('./data/metrics/second_step/opus3.csv')
print(OpusModel_data["precision"].mean())

95.8864719789491


In [24]:
Haiku3_data = pd.read_csv('./data/metrics/second_step/haiku_3.csv')
print(Haiku3_data["precision"].mean())

96.5543244766929


In [25]:
Sonet3_data = pd.read_csv('./data/metrics/second_step/sonet3.csv')
print(Sonet3_data["precision"].mean())

95.29168838534972


In [26]:
small_llama_data = pd.read_csv('./data/metrics/second_step/small_llam.csv')
print(small_llama_data["precision"].mean())

83.7409019575196


In [27]:
big_llama_data = pd.read_csv('./data/metrics/second_step/big_llama3.csv')
print(big_llama_data["precision"].mean())

94.76469282497314


In [29]:
mistral_data = pd.read_csv('./data/metrics/second_step/big_mistral.csv')
print(mistral_data["precision"].mean())

90.27394385669223


In [ ]:
last_llama = pd.read_csv('./data/metrics/second_step/big_llama3_1.csv')
print(last_llama["precision"].mean())